In [5]:
import math
import numpy as np
import collections

In [6]:
#假设有以下行为数据:
user_item_time_dict = {
    '用户A': [('新闻1', 1.6), ('新闻2',2.4), ('新闻6', 1.0), ('新闻7', 2.5), ('新闻6', 1.9), ('新闻7', 0.4)],  # 点击了商品1和商品2
    '用户B': [('新闻1', 0.9), ('新闻3', 1.3)],
    '用户C': [('新闻99', 2.0), ('新闻1', 0.5), ('新闻3', 2.1)],
    '用户D': [('新闻3', 1.4), ('新闻4', 1.1), ('新闻6', 1.0), ('新闻3', 0.4)],
    '用户E': [('新闻9', 2.1), ('新闻3', 1.8)],
    '用户F': [('新闻1', 2.0), ('新闻3', 1.7)],
    '用户G': [('新闻9', 1.5), ('新闻7', 2.1)], 
    '用户H': [('新闻7', 1.6), ('新闻3', 2.2)],  
}


在没有得到相似性矩阵之前，我们可以对数据简单分析：
比如：
- 新闻1后面经常跟着新闻3，所以新闻1和新闻3的权重应该要大于新闻1和其他新闻的权重
- 只有一个新闻2，新闻2和新闻6的相关性应该比新闻2和新闻1的相关性高，因为新闻2后面跟了两个新闻6（当然也跟了两个新闻7）

📌任务一：不考虑新闻内容 去得到新闻相似度矩阵

不考虑内容，怎么建立新闻之间的相似性呢？---->基于规则
1. 用户点击的时间权重 ---- 同一个用户，两篇文章点击间隔时间越长，越不相关，`click_time_weight`权重越小
2. 用户点击的顺序权重 ---- 同一个用户，两篇文章点击间隔越长(或者理解为相对距离较大)，越不相关

In [7]:
i2i_sim = {} #保存新闻之间的相似性
item_cnt = collections.defaultdict(int)

for user,item_time_list in user_item_time_dict.items():
    for loc1,(i,time_i) in enumerate(item_time_list):
        i2i_sim.setdefault(i,{})
        item_cnt[i] += 1
        for loc2,(j,time_j) in enumerate(item_time_list):
            if i==j:
                continue
            #时间权重
            click_time_weight = np.exp(0.8**np.abs(time_i-time_j))#∈[1,e]
            #顺序权重
            loc_alpha = 1.0 if loc2>loc1 else 0.7
            loc_weight = loc_alpha * (0.9**(np.abs(loc2-loc1)-1))
            #最终权重
            i2i_sim[i].setdefault(j,0)
            i2i_sim[i][j] += loc_weight*click_time_weight / math.log(len(item_time_list) + 1)#除以log(),是为了减少活跃用户的权重
for i, related_items in i2i_sim.items():
    for j, wij in related_items.items():
        i2i_sim[i][j] = wij / math.sqrt(item_cnt[i] * item_cnt[j])
i2i_sim

{'新闻1': {'新闻2': 0.5931159611717185,
  '新闻6': 0.5957274652784682,
  '新闻7': 0.4169767138246315,
  '新闻3': 1.1420539537573482,
  '新闻99': 0.5163788220092422},
 '新闻2': {'新闻1': 0.41518117282020295,
  '新闻6': 1.204538013991482,
  '新闻7': 0.9701355065018669},
 '新闻6': {'新闻1': 0.4170092256949279,
  '新闻2': 0.8431766097940373,
  '新闻7': 1.1440380504432999,
  '新闻3': 0.538343920879418,
  '新闻4': 0.6676886445636208},
 '新闻7': {'新闻1': 0.291883699677242,
  '新闻2': 0.6790948545513068,
  '新闻6': 0.9822652253203883,
  '新闻9': 0.5402334687943983,
  '新闻3': 0.41252419369327964},
 '新闻3': {'新闻1': 0.7994377676301436,
  '新闻99': 0.45671668503165624,
  '新闻4': 0.9463652973962783,
  '新闻6': 0.532164875384628,
  '新闻9': 0.433873184783495,
  '新闻7': 0.2887669355852957},
 '新闻99': {'新闻1': 0.7376840314417747, '新闻3': 0.6524524071880803},
 '新闻4': {'新闻3': 0.91601503340005, '新闻6': 0.9538409208051726},
 '新闻9': {'新闻3': 0.6198188354049929, '新闻7': 0.7717620982777119}}

📌任务二：根据新闻内容去建立新闻相似度矩阵
- 读文件（360000篇新闻内容，每一篇对应251维的向量）

- 向量归一化

- 利用Faiss检索得到每篇新闻和其他新闻的topk相似性、topk新闻的id

- 返回相似性词典

（一）数据部分我们模拟生成16篇文章的向量，每个向量10维，如下：


In [14]:
import pandas as pd
import numpy as np
import faiss

# 生成16个10维的随机向量
random_vectors = np.random.rand(16, 10)
# 创建DataFrame
df_vectors = pd.DataFrame(random_vectors, columns=[f"Dim_{i+1}" for i in range(10)])
df_vectors.head()


,Dim_1,Dim_2,Dim_3,Dim_4,Dim_5,Dim_6,Dim_7,Dim_8,Dim_9,Dim_10
0,0.073402,0.322475,0.655846,0.745564,0.974856,0.001904,0.344626,0.499609,0.822746,0.239762
1,0.676631,0.809446,0.042787,0.671026,0.821358,0.007901,0.146246,0.318660,0.142671,0.927429
2,0.250118,0.056921,0.639428,0.540723,0.363758,0.873752,0.820639,0.129606,0.476590,0.038671
3,0.668918,0.242803,0.910671,0.619329,0.503281,0.036067,0.150108,0.534444,0.749239,0.049436
4,0.480549,0.969858,0.823346,0.106325,0.096847,0.881640,0.055947,0.678211,0.816086,0.526621


In [ ]:
item_emb_cols = df_vectors.columns
item_emb_np = np.ascontiguousarray(df_vectors.values,dtype=np.float32)

In [ ]:
#归一化
item_emb_np = item_emb_np / np.linalg.norm(item_emb_np, axis=1, keepdims=True)

# 建立faiss索引
item_index = faiss.IndexFlatIP(item_emb_np.shape[1])
item_index.add(item_emb_np)
# 相似度查询，给每个索引位置上的向量返回topk个item以及相似度
sim, idx = item_index.search(item_emb_np, 3) # 返回的是列表,返回3个相似度最高的

In [21]:
i2i_emb_sim = {}
for n,sim_values,rele_idxs in zip(range(1,16),sim,idx):
    i2i_emb_sim.setdefault(n,{})
    for rele_idx, sim_value in zip(rele_idxs[1:], sim_values[1:]):
        i2i_emb_sim[n][rele_idx] = sim_value
i2i_emb_sim

{1: {3: 0.8766532, 11: 0.7732587},
 2: {14: 0.83587056, 7: 0.8355814},
 3: {14: 0.7663307, 15: 0.74937594},
 4: {0: 0.8766532, 11: 0.8165651},
 5: {12: 0.86976963, 14: 0.83409804},
 6: {6: 0.89792454, 11: 0.88355124},
 7: {14: 0.9372531, 12: 0.90049255},
 8: {14: 0.9236369, 9: 0.8715523},
 9: {6: 0.87652004, 14: 0.86646056},
 10: {7: 0.8715523, 11: 0.8632221},
 11: {12: 0.905044, 6: 0.8936068},
 12: {5: 0.88355124, 9: 0.8632221},
 13: {10: 0.905044, 6: 0.90049255},
 14: {10: 0.854641, 7: 0.82095826},
 15: {6: 0.9372531, 7: 0.9236369}}

📌任务三:根据每个用户的历史点击结果 做召回
这里在召回的时候，也是用了关联规则的方式：

📌任务四: 评估召回结果
- 计算命中数： 对于每个用户，检查其最后一次点击的文章是否在召回的前 k 个文章中，如果在，则算作一次命中。
- 计算击中率： 用命中数除以总用户数，得到击中率。